In [2]:
# do not forget to set the parameters topn_detection, topn_correction, and method

import pandas as pd
import numpy as np
import re
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AdamW
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import pickle
import ast
import statistics as s
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
import nltk
from transformers import pipeline
import copy
from nltk.corpus import stopwords

/home/nvanthof/.local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# load word2vec model
#w2v_model = ...
# load BERT model
#BERT_model = ...
# load dataframe
#df = ...

tokenizer = BertTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
BERT_model = BertForMaskedLM.from_pretrained("GroNLP/bert-base-dutch-cased")
#w2v_model.intersect_word2vec_format(r"combined-160.txt", binary=False, lockf=1.0)
# https://github.com/clips/dutchembeddings

df = pd.read_csv('preprocessed_df100.csv')

In [4]:
def finetune_word2vec(train, window=5):
    sentences = train.split('.')
    tokenizer = RegexpTokenizer(r'\w+')
    sentences = [tokenizer.tokenize(i) for i in sentences]
    total_examples = len(sentences)
    
    model_w2v = Word2Vec(size=160, min_count=1, window=window)
    model_w2v.build_vocab(sentences)
    total_examples = model_w2v.corpus_count
    model = KeyedVectors.load_word2vec_format(r"combined-160.txt", binary=False)
    model_w2v.build_vocab([list(model.vocab.keys())], update=True)
    model_w2v.intersect_word2vec_format(r"combined-160.txt", binary=False, lockf=1.0)
    model_w2v.train(sentences, total_examples=total_examples, epochs=model_w2v.iter)
    return model_w2v

train = df['gt text'][0]
word2vec_model = finetune_word2vec(train)

/home/nvanthof/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  del sys.path[0]


In [5]:
with open('all_lists_tokens', 'rb') as f:
    all_lists_tokens = pickle.load(f)
    
all_lists_token = ast.literal_eval(all_lists_token)
vocab_BERT, vocab_word2vec, hist_expressions, modern_vocab, dictionary = all_lists_tokens

FileNotFoundError: [Errno 2] No such file or directory: 'all_lists_tokens'

In [ ]:
ldef correct_sorted(candidates, sim_or_probs, LD): # sorts first by LD, then by similarity/probability
    paired_sorted = sorted(zip(LD,sim_or_probs,candidates),key = lambda x: (x[0],x[1]), reverse=True)
    LD,sim_or_probs,candidates = zip(*paired_sorted)
    correction = candidates[0]
    return correction
    
def correct_calculated(candidates, sim_or_probs, LD): # calculates a score from LD and normalised similarity/probability
    inv_LD = 1 - LD
    sim_or_probs = np.array(sim_or_probs)
    sim_or_probs = np.interp(sim_or_probs, (sim_or_probs.min(), sim_or_probs.max()), (0, 1)).tolist()
    score = sim_or_probs / inv_LD
    zipped_pairs = zip(score.tolist(), candidates)
    sorted_by_score = [x for _, x in sorted(zipped_pairs, reverse=True)]
    correction = sorted_by_score[0]
    return correction

def remove_stopwords(candidates, cosine, LD):
    #nltk.download('stopwords')
    stop_words = set(stopwords.words('dutch'))
    candidates_nostopwords = []
    cosine_nostopwords = []
    LD_nostopwords = []
    for i in range(len(candidates)):
        if candidates[i] not in stop_words:
            candidates_nostopwords.append(candidates[i])
            cosine_nostopwords.append(cosine[i])
            LD_nostopwords.append(LD[i])
    LD_nostopwords = np.array(LD_nostopwords)
    return candidates_nostopwords, cosine_nostopwords, LD_nostopwords

In [ ]:
# lists of all FN, TN, FP, TPs detection:
homonyms_detection_list_w2c = [[],[],[],[]]
homonyms_detection_context_list_w2c = [[],[],[],[]]
histexp_detection_list_w2c = [[],[],[],[]]
histexp_detection_context_list_w2c = [[],[],[],[]]
OOV_detection_list_w2c = [[],[],[],[]]
OOV_detection_context_list_w2c = [[],[],[],[]]
infreq_detection_list_w2c = [[],[],[],[]]
infreq_detection_context_list_w2c = [[],[],[],[]]
RWE_detection_list_w2c = [[],[],[],[]]
RWE_detection_context_list_w2c = [[],[],[],[]]
all_detection_list_w2c = [[],[],[],[]]

# list of all right / wrong correction
homonyms_correction_list_w2c = [[],[]]
homonyms_correction_context_list_w2c = [[],[]]
histexp_correction_list_w2c = [[],[]]
histexp_correction_context_list_w2c = [[],[]]
OOV_correction_list_w2c = [[],[]]
OOV_correction_context_list_w2c = [[],[]]
infreq_correction_list_w2c = [[],[]]
infreq_correction_context_list_w2c = [[],[]]
RWE_correction_list_w2c = [[],[]]
RWE_correction_context_list_w2c = [[],[]]
all_correction_list_w2c = [[],[]]

#list of outputs corrected texts


In [ ]:


all_detection_nums_w2c = [0,0,0,0]

def calculate_result(predicted_error, actual_error):
    if actual_error == True:
        if predicted_error == True: # TP
            result = 'TP'
        if predicted_error == False: # FN
            result = 'FN'
    if actual_error == False:
        if predicted_error == True: # FP
            result = 'FP'
        if predicted_error == False: # TN
            result = 'TN'
    return result

def special_tokens_detection_word(gt_word, detection_list_w2c, all_lists_tokens, result): 
    # check if word is homonym
    if gt_word in all_lists_tokens[0]:
        homonym = True
    # check if word is OOV
    
    # check if word is historical expression
    
    # check if word is infrequent
    
    # check if word is RWE
    
    if result == 'TP': # actual error == True, predicted error == True
        
    elif result == 'FN': # actual error == True, predicted error == False
        
    elif result == 'FP': # actual error == False, predicted error == True
         
    elif result == 'TN': # actual error == False, predicted error == False
     
    
def special_tokens_detection_context(predicted_error, word_GT, word_OCR, context_GT):
    
    
def special_tokens_correction_word(predicted_error, word_GT, word_OCR):
    
    
def special_tokens_correction_context(predicted_error, word_GT, word_OCR, context_GT):
    
    


In [ ]:
#detection test word2vec
def detection_and_correction_word2vec(row, w2v_model, ocr_names, window=5, topn_detection=1000, topn_correction=1000, all_lists_token):
    if row['set'] != 'test':
        return np.nan
    else:
        print('detection')
        identifier = row['identifier']
        OCR_text = row['aligned_OCR_sentences']
        GT_text = row['aligned_GT_sentences']
        OCR_text = ast.literal_eval(OCR_text)
        GT_text = ast.literal_eval(GT_text)
        OCR_text = list_merger(OCR_text)
        GT_text = list_merger(GT_text)
        window_range = list(range(0,window))
        window_range = np.array(window_range) - ((window - 1) / 2)

        # keep track of performance
        homonyms_detection_list_w2c = [0,0,0,0]
        homonyms_detection_context_list_w2c = [0,0,0,0]
        histexp_detection_list_w2c = [0,0,0,0]
        histexp_detection_context_list_w2c = [0,0,0,0]
        OOV_detection_list_w2c = [0,0,0,0]
        OOV_detection_context_list_w2c = [0,0,0,0]
        infreq_detection_list_w2c = [0,0,0,0]
        infreq_detection_context_list_w2c = [0,0,0,0]
        RWE_detection_list_w2c = [0,0,0,0]
        RWE_detection_context_list_w2c = [0,0,0,0]
        all_detection_list_w2c = [0,0,0,0]
        
        detection_list_w2c = [homonyms_detection_list_w2c, histexp_detection_list_w2c, OOV_detection_list_w2c, infreq_detection_list_w2c, RWE_detection_list_w2c, all_detection_list_w2c]
        detection_list_context_w2c = [homonyms_detection_context_list_w2c, histexp_detection_context_list_w2c, OOV_detection_context_list_w2c, infreq_detection_context_list_w2c, RWE_detection_context_list_w2c]

        
        for i in range(len(OCR_text)):
            try:
                if (OCR_text[i] in ocr_names) or (OCR_text[i].isalpha() == False) or (len(OCR_text[i]) <= 2):
                    continue
                context = []
                GT_context = []
                for j in window_range:
                    if (i+j >= 0) and (i+j < len(OCR_text)) and j != 0:
                        context.append(OCR_text[i+int(j)])
                        GT_context.append(OCR_text[i+int(j)])
                    else:
                        pass
                candidates = []
                cosines = []
                # calculate positions detection task
                for prediction in w2v_model.predict_output_word(context, topn=topn_detection):
                    candidates.append(prediction[0])
                    cosines.append(prediction[1]) 
                # remove punctuation except for hyphen from candidates
                candidates = [re.sub(r'[^\w\d\s\-]+', '', x) for x in candidates]
                # determine if token is predicted error or not
                if OCR_text[i] in candidates:
                    predicted_error = False
                elif OCR_text[i] not in candidates:
                    predicted_error = True
                # determine if token is actual error or not
                if OCR_text[i] =! GT_text[i]:
                    actual_error = True
                elif OCR_text[i] == GT_text[i]:
                    actual_error = False
                result = calculate_result(predicted_error, actual_error)
                # evaluate detection
                detection_list_w2c = special_tokens_detection_word(detection_list_w2c, all_lists_token, result)
                
                
                
                if error == True:
                    
                    # try two correction methods
                    # first calculate the normalized LDs:
                    LD = np.array([fuzz.ratio(OCR_text[i], word)/100 for word in candidates])
                    # try sorting method
                    correction = correct_sorted(candidates, cosines, LD)
                    if correction == GT_text[i]:
                        rights_correct_sorted += 1
                    elif correction != GT_text[i]:
                        wrongs_correct_sorted += 1
                    print('OCR:', OCR_text[i])
                    print('correction sorted:', correction)
                    # try again the sorting methods, but without stopwords
                    candidates_nostopwords, cosine_nostopwords, LD_nostopwords = remove_stopwords(candidates, cosines, LD)
                    correction = correct_sorted(candidates_nostopwords, cosine_nostopwords, LD_nostopwords)
                    if correction == GT_text[i]:
                        rights_correct_sorted_nosw += 1
                    elif correction != GT_text[i]:
                        wrongs_correct_sorted_nosw += 1
                    # try score calculation method
                    correction = correct_calculated(candidates, cosines, LD)
                    if correction == GT_text[i]:
                        rights_correct_calculated += 1
                    elif correction != GT_text[i]:
                        wrongs_correct_calculated += 1
                    print('OCR:', OCR_text[i])
                    print('correction score:', correction)
                elif error == False:
                    # where the non error is in the detection list
                    non_error_positions_doc.append(position)
                # calculate positions correction task
                if error == True:
                    try:
                        right_token_position = candidates.index(GT_text[i])
                        right_token_positions_doc.append(right_token_position)
                    except ValueError:
                        right_token_positions_doc.append(topn_detection)
                    right_token_positions.append(right_token_positions_doc)
            except:
                pass
            
            
    #return error_positions, non_error_positions
    error_positions.append(error_positions_doc)
    non_error_positions.append(non_error_positions_doc)
    right_token_positions.append(right_token_positions_doc)
    
    # add performance to the list
    rights_correct_sorted_list.append(rights_correct_sorted)
    wrongs_correct_sorted_list.append(wrongs_correct_sorted)
    rights_correct_sorted_nosw_list.append(rights_correct_sorted_nosw)
    wrongs_correct_sorted_nosw_list.append(wrongs_correct_sorted_nosw)
    rights_correct_calculated_list.append(rights_correct_calculated)
    wrongs_correct_calculated_list.append(wrongs_correct_calculated)
    
    return identifier, error_positions, non_error_positions, right_token_positions, rights_correct_sorted_list, wrongs_correct_sorted_list, rights_correct_sorted_nosw_list, wrongs_correct_sorted_nosw_list, rights_correct_calculated_list, wrongs_correct_calculated_list   
    
#for index, row in df.iterrows():
#    detection_word2vec(row)
# df.loc[70]
fake_test_list_GT_aligned = """Een koekenpan of kortweg pan is een platte pan met een lang handvat.
De pan ontleent zijn naam aan het feit dat in zo'n pan pannenkoeken worden gebakken. Ook ander voedsel, zoals vlees, wordt in een koekenpan gebraden"""
fake_test_list_OCR_aligned = """Een hoekenpan of kortweg pan is een platte pan met een hang handvat.
De pan ontleent zijn naam haan het feit dat in zo'n pan pannenkoeken horden gebakken. Ook ander voedsel, zoals vlees, word in een hoekenpan gebraden"""
fake_test_list_GT_aligned = fake_test_list_GT_aligned.split('.')
fake_test_list_OCR_aligned = fake_test_list_OCR_aligned.split('.')
fake_test_list_GT_aligned = [x.split(' ') for x in fake_test_list_GT_aligned]
fake_test_list_OCR_aligned = [x.split(' ') for x in fake_test_list_OCR_aligned]
d = {'identifier': ['111'], 'aligned_OCR_sentences': [str(fake_test_list_OCR_aligned)], 'aligned_GT_sentences': [str(fake_test_list_GT_aligned)], 'set': ['val']}
df_probeer = pd.DataFrame(data=d)


def perform_task(row, w2v_model, ocr_names, window=5):
    if row['set'] != 'test':
        return np.nan
    else:
        print('detection')
        identifier = row['identifier']
        OCR_text = row['aligned_OCR_sentences']
        GT_text = row['aligned_GT_sentences']
        OCR_text = ast.literal_eval(OCR_text)
        GT_text = ast.literal_eval(GT_text)
        OCR_text = list_merger(OCR_text)
        GT_text = list_merger(GT_text)
        window_range = list(range(0,window))
        window_range = np.array(window_range) - ((window - 1) / 2)

        # keep track of performance
        homonyms_detection_list_w2c = [0,0,0,0]
        homonyms_detection_context_list_w2c = [0,0,0,0]
        histexp_detection_list_w2c = [0,0,0,0]
        histexp_detection_context_list_w2c = [0,0,0,0]
        OOV_detection_list_w2c = [0,0,0,0]
        OOV_detection_context_list_w2c = [0,0,0,0]
        infreq_detection_list_w2c = [0,0,0,0]
        infreq_detection_context_list_w2c = [0,0,0,0]
        RWE_detection_list_w2c = [0,0,0,0]
        RWE_detection_context_list_w2c = [0,0,0,0]
        all_detection_list_w2c = [0,0,0,0]
        all_detection_context_list_w2c = [0,0,0,0]
        
        for i in range(len(OCR_text)):
            try:
                if (OCR_text[i] in ocr_names) or (OCR_text[i].isalpha() == False) or (len(OCR_text[i]) <= 2):
                    continue
                context = []
                GT_context = []
                for j in window_range:
                    if (i+j >= 0) and (i+j < len(OCR_text)) and j != 0:
                        context.append(OCR_text[i+int(j)])
                        GT_context.append(OCR_text[i+int(j)])
                    else:
                        pass
            
    
    